In [334]:
import pandas as pd
from collections import defaultdict
import re

from bs4 import BeautifulSoup
import requests
import time, os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import time

# Set up Firefox driver
geckodriver = "/Applications/geckodriver"
os.environ['webdriver.firefox.driver'] = geckodriver

In [335]:
# t&H(YwSS8:V5E?c

In [336]:
# Provide job alert URL's with descriptions
job_alerts = {#'Small Company Remote Analyst US' : 'https://www.linkedin.com/jobs/search/?f_C=2029%2C68543%2C2658876%2C3477522%2C3806620%2C6459033%2C18696467%2C312538%2C3765675%2C3293310%2C1253671%2C2908815%2C3881453%2C1835869%2C2029368%2C279570%2C2503130%2C7944399%2C1110454%2C2748615%2C3640016%2C6591023%2C10643112%2C18348748%2C2464624%2C5233302%2C10345478%2C1120204%2C14041470%2C18917297%2C19006475%2C19023241%2C19852792%2C2120791%2C2544439%2C2575799%2C27116184%2C2762432%2C28639174%2C3317416%2C35539062%2C3818105%2C3832515%2C3896605%2C6633924%2C6653417%2C72307365&f_WT=2&geoId=103644278&keywords=analyst&location=United%20States',
              #'Big Company Remote Analyst US' : 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=United%20States&locationId=&geoId=103644278&f_TPR=&f_C=1586%2C3185%2C2382910%2C2532259%2C96622%2C20226%2C1028%2C10667%2C1441%2C1063%2C1815218%2C2672915%2C675562%2C3608%2C15088102%2C732400%2C13990%2C54178%2C1337%2C19022%2C19857%2C1512%2C22688%2C1033%2C3254263%2C1035%2C1480%2C206993%2C1463%2C165697%2C2748615%2C1482%2C166000%2C2029%2C822535%2C162479%2C165158%2C207470%2C1038%2C1103%2C1307%2C1371%2C147977%2C16140%2C2271%2C2658876%2C2908815%2C309694%2C3595%2C51941%2C6633924%2C665858%2C76987811&f_JT=F%2CC&f_WT=2&position=1&pageNum=0',
              #'ML Remote US' : 'https://www.linkedin.com/comm/jobs/search?savedSearchId=1726769010&alertAction=viewjobs&f_TPR=a1646969285-&savedSearchAuthToken=1%26AQEw-KRlCuTsuAAAAX-bEW9MIlDcL9bCszsay7OgwQiPRDn-8AsPZe_DmrL1NW3J6Be7zJYC_4PDtKpPaCtO5fu99p90e9AMwGz8ikB6k1qVuburQhBZ9afcVyS-ChJdV2VlCOwGGs0B6p5fqoK8jfuSMlQbFX3fXA5E9WM_AAzGSlT3JxYBKggelYnNuVBR3iEqT91KyLfJwY87_6ujI4sWzymir3IVI2X5qKXiOsyM9W-cvroybKhaGtmj6B2pZwgtBtOlReSXGSaPRp_w7mDkzXSpfcXj47GRk3vRl5_Bwoe05lY%26Ab1rT8684_eJGFHi02PCijuFMJQR&midToken=AQGOAaXZ2Mv6pw&midSig=0nOMaFZ6zM_W81&trk=eml-email_job_alert_confirmation_01-job_alert-0-header_mercado&trkEmail=eml-email_job_alert_confirmation_01-job_alert-0-header_mercado-null-39xk3l%7El0vv2kjr%7Ep3-null-jobs%7Ejserp%7Esearch&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_confirmation_01%3BHzdSji8SSyaBBISxzuCUSA%3D%3D',
              #'Data Analyst Seattle' : 'https://www.linkedin.com/jobs/search/?f_E=2%252C3&f_JT=F%252CC&f_WT=2&geoId=103644278&keywords=data%20analyst%20seattle&location=United%20States',
              #'Top Small Companies to Work Data Analyst' : 'https://www.linkedin.com/jobs/search/?f_C=1866670%2C97475%2C1253671%2C5243263%2C3806620%2C8495%2C1030330%2C10879774%2C3677944%2C3112877%2C3782894%2C3877505%2C1175268%2C6459033%2C25016898%2C25237%2C2973445%2C377671%2C41771%2C425337%2C510820%2C1165219%2C156534%2C16162690%2C17895640%2C222181%2C2820889%2C28669481%2C2908815%2C2977830%2C3056596%2C33189811%2C34957318%2C3961599%2C40026%2C5073200%2C5225921%2C631652%2C68648187%2C71655160%2C74157&f_WT=2&keywords=data%20analyst',
              #'Great Small Places to Work' : 'https://www.linkedin.com/jobs/search/?f_C=3126445%2C147977%2C18373097%2C5101804%2C11220691%2C3640016%2C137338%2C7587729%2C18880849%2C224232%2C10526429%2C18226665%2C1637984%2C3809194%2C308865%2C222181%2C34224678%2C231741%2C9451029%2C3738695%2C792882%2C2582861%2C21837%2C3089380%2C210676%2C5243263%2C2225282%2C15175997%2C863947%2C14059116%2C2029368%2C807257&f_WT=2&geoId=103644278&keywords=analyst&location=United%20States&sortBy=DD',
              'More Analyst Jobs' : 'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&f_EA=%5B%22true%22%5D&f_F=%5B%22anls%22%2C%22rsch%22%2C%22cnsl%22%2C%22fin%22%2C%22mrkt%22%2C%22bd%22%2C%22prdm%22%5D&f_JT=%5B%22F%22%5D&f_T=%5B%22340%22%2C%2229%22%2C%22194%22%2C%2226610%22%2C%222463%22%5D&f_WT=%5B%222%22%5D&geoId=103644278&keywords=data%20analyst%20-%22edward%20jones%22%20-pwc%20-merkle&location=United%20States'
             }


In [337]:
# Set up Selenium driver
driver = webdriver.Firefox()

In [338]:
# Set up empty dataframe to load job data
job_data = pd.DataFrame(columns=['ID',
                                 'Date',
                                 'Company',
                                 'Title',
                                 'Location',
                                 'Description',
                                 'Link'
                                ])

In [339]:
for key in job_alerts:
    
    print(key)
    
    # Go to web page
    driver.get(job_alerts[key]);
    time.sleep(5)
    
    # Find job count
    attempt = 0
    stop = False
    
    while (stop == False) & (attempt < 5):
        try: 
            driver.get(job_alerts[key]);
            time.sleep(5)
            job_count = int(driver.find_element('css selector', 'small.display-flex').get_attribute('innerText').split(' ')[0].replace(',', ''))
            #job_count = int(''.join(re.findall('\d+', driver.find_element('css selector', 'h1>span').get_attribute('innerText'))))
            job_lists = driver.find_element('css selector', '.jobs-search-results__list')
            jobs = job_lists.find_elements('tag name', 'li')
            stop = True
        except:
            attempt += 1
            print('Attempt #' + str(attempt) + ' failed')
            next
    
    print('Job Count: ' + str(job_count))
    
    if attempt == 5:
        print('Scrape failed for ' + key)
        next

    # Check for pagination    
    try:
        final_page = driver.find_element('class name', 'jobs-search-two-pane__pagination')\
                        .find_element('tag name', 'div')\
                        .find_element('tag name', 'ul')\
                        .find_elements('tag name', 'li')[-1]\
                        .get_attribute('innerText')
    
        start_list = [x * 25 for x in range(0, int(final_page))]
        pages = True
    except:
        pages = False
    
    # Set up lists of job details to be loaded to dataframe
    job_id= []
    job_title = []
    company_name = []
    location = []
    date = []
    job_link = []
    
    if pages:
        # Go to each page to get job details
        for start in start_list:
            try:
                driver.get(job_alerts[key] + '&start=' + str(start))
                time.sleep(3)

                # Scroll to footer to load more jobs
                list_items = driver.find_element('css selector', '.jobs-search-results__list').find_elements('tag name', 'li')

                for i in range(1,10):
                    driver.execute_script("arguments[0].scrollIntoView();", list_items[int(len(list_items) * i / 10)])
                    time.sleep(2)

                driver.execute_script("arguments[0].scrollIntoView();", list_items[-1])
            except:
                pass

            html = driver.page_source
            soup = BeautifulSoup(html, 'html5lib')

            jobs = soup.find('ul', {'class': 'jobs-search-results__list list-style-none'})\
                            .find_all('li', {'class': 'jobs-search-results__list-item occludable-update p0 relative ember-view'})

            for job in jobs:
                try:
                    job_id0 = job.find('div').find('div')['data-job-id']
                    job_title0 = job.find('div', {'class': 'full-width artdeco-entity-lockup__title ember-view'}).text.strip()
                    company_name0 = job.find('div', {'class': 'artdeco-entity-lockup__subtitle ember-view'}).text.strip()
                    location0 = ' - '.join([x.strip() for x in job.find('div', {'class': 'artdeco-entity-lockup__caption ember-view'}).text.split('\n') if x.strip()])
                    job_link0 = 'https://www.linkedin.com/' + job.find('div', {'class': 'full-width artdeco-entity-lockup__title ember-view'}).find('a', href=True)['href']
                    date0 = ''

                    job_id.append(job_id0)
                    job_title.append(job_title0)
                    company_name.append(company_name0)
                    location.append(location0)
                    date.append(date0)
                    job_link.append(job_link0)
                
                except:
                    next
    else:
        # Scroll to the bottom of all jobs
        i = 1

        while (len(jobs) < job_count and i < 100): 
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

            try:
                driver.find_element('xpath', '/html/body/div[1]/div/main/section[2]/button').click()
                time.sleep(3)
            except:
                time.sleep(3)

            job_lists = driver.find_element('css selector', '.jobs-search-results__list')
            jobs = job_lists.find_elements('tag name', 'li')

            i += 1

        # Set up list of jobs on page
        job_lists = driver.find_element('css selector', '.jobs-search-results__list')
        jobs = job_lists.find_elements('tag name', 'li')

        for job in jobs:
            for job in jobs:
                try:
                    job_id0 = job.find_element('css selector', 'div').find_element('css selector', 'div').get_attribute('data-job-id')
                    job_id.append(job_id0)
                except:
                    jobs.remove(jobs)
                    next

                job_title0 = job.find_element('css selector', 'div').find_element('css selector', 'div').find_element('css selector', 'div').find_elements('css selector', 'div')[1].find_elements('css selector', 'div')[0].get_attribute('innerText')
                job_title.append(job_title0)

                company_name0 = job.find_element('css selector', 'div').find_element('css selector', 'div').find_element('css selector', 'div').find_elements('css selector', 'div')[1].find_elements('css selector', 'div')[1].get_attribute('innerText')
                company_name.append(company_name0)
                location0 = ' - '.join(job.find_element('css selector', 'div').find_element('css selector', 'div').find_element('css selector', 'div').find_elements('css selector', 'div')[1].find_elements('css selector', 'div')[2].get_attribute('innerText').split('\n'))
                location.append(location0)

                try:
                    date0 = job.find_element('css selector', 'div>div>time').get_attribute('datetime')
                    date.append(date0)
                except:
                    date.append('')

                job_link0 = job.find_element('css selector', 'div').find_element('css selector', 'div').find_element('css selector', 'div').find_elements('css selector', 'div')[1].find_elements('css selector', 'a')[0].get_attribute('href')
                job_link.append(job_link0)
    
    min_length = min(len(job_id),
                     len(job_title),
                     len(company_name),
                     len(date),
                     len(location),
                     len(job_link)
                    )
    
    job_id = job_id[0 : min_length - 1]
    job_title = job_title[0 : min_length - 1]
    job_link = job_link[0 : min_length - 1]
    location = company_name[0 : min_length - 1]
    company_name = company_name[0 : min_length - 1]
    date = date[0 : min_length - 1]
    job_link = job_link[0 : min_length - 1]
                         

    # Get job description and additional details from each job page to load to dataframe
    jd = []

    for link in job_link:
        driver.get(job_link[0])
        html = driver.page_source
        soup = BeautifulSoup(html, 'html5lib')

        try:
            jd0 = soup.find('div', {'id': "job-details"}).text.strip()
            jd.append(jd0)
        except AttributeError:
            jd.append('')

    # Create dataframe for current key in dictionary and amend to the full dataframe
    key_data = pd.DataFrame({'ID': job_id,
                             'Date': date,
                             'Company': company_name,
                             'Title': job_title,
                             'Location': location,
                             'Description': jd,
                             'Link': job_link
                            })
    
    print('Length: ' + str(len(key_data)))
    job_data = job_data.append(key_data)

More Analyst Jobs
Job Count: 1208
Length: 372


In [340]:
# Transorm job description column to lower case
job_data['Description'] = job_data['Description'].str.lower()
job_data['Description'] = job_data['Description'].str.replace('\n', '')

In [341]:
# Create binary columns to tag whether key phrases are included in job descriptions
job_data['Remote'] = job_data['Description'].str.contains('remote').astype(int)
job_data['Python'] = job_data['Description'].str.contains('python').astype(int)
job_data['Programming'] = job_data['Description'].str.contains('programming').astype(int)
job_data['ML'] = job_data['Description'].str.contains('machine learning').astype(int)
job_data['SQL'] = job_data['Description'].str.contains('sql').astype(int)
job_data['BI'] = job_data['Description'].str.contains('business intelligence').astype(int)
job_data['Tableau'] = job_data['Description'].str.contains('tableau').astype(int)
job_data['AB'] = job_data['Description'].str.contains('ab test | a/b test').astype(int)
job_data['Forecasting'] = job_data['Description'].str.contains('forecast').astype(int)
job_data['Revenue'] = job_data['Description'].str.contains('revenue').astype(int)

job_data['Total Hits'] = job_data.loc[:,'Remote':].sum(axis=1)

In [342]:
# Rename columns
job_data = job_data[['ID',
                     'Company',
                     'Title',
                     'Date',
                     'Location',
                     'Link',
                     'Total Hits',
                     'Remote',
                     'Python',
                     'Programming',
                     'ML',
                     'SQL',
                     'BI',
                     'Tableau',
                     'AB',
                     'Forecasting',
                     'Revenue',
                     'Description'
                    ]]

In [343]:
job_data.to_csv('job_data_combined.csv')